In [1]:
import torch
from torch import nn as nn
from models.nac import NAC
from models.nalu import NALU
from utilities.train_utils import get_batches,\
get_eval_loss, get_eval_preds
from torch.optim import Adam, SGD
import numpy as np
from tqdm import tqdm
from tqdm import tqdm_notebook
import time

In [2]:
def make_data(min_val, max_val, num_obs, op):
    data = np.random.uniform(min_val, max_val, size=(num_obs, 2))
    if op == '+':
        targets = data[:, 0] + data[:, 1]
    elif op == '-':
        targets = data[:, 0] - data[:, 1]
    elif op == '*':
        targets = data[:, 0] * data[:, 1]
    elif op == '/':
        targets = data[:, 0] / data[:, 1]
    elif op == 'sqrt':
        data = np.random.uniform(min_val, max_val, size=(num_obs, 1))
        targets = np.sqrt(data)
    return data, targets

In [3]:
eps = 1e-12
X_train, Y_train = make_data(10, 20, 10000, '/')

X_valid, Y_valid = make_data(10, 20, 1000, '/')

X_test, Y_test = make_data(0, 30, 5000, '/')

In [4]:
def accuracy_score(preds, targets, tol=1e-3):
    preds = preds.cpu().numpy().flatten()
    targets = targets.cpu().numpy().flatten()
    accuracy = np.isclose(preds, targets, rtol=tol)
    accuracy = accuracy.astype(np.int32).mean()
    return accuracy * 100

In [19]:
batch_size = 16
patience = 15
running_patience = 5
checkpoint = 'best_model.sav'
print_every = 50
num_epochs = 500000
running_batch = 0
running_loss = 0
min_loss = float('inf')
use_gpu = torch.cuda.is_available()

criterion = nn.SmoothL1Loss()

model = NALU(2, 1)

if use_gpu:
    model = model.cuda()
optimizer = Adam(model.parameters())

for epoch in range(num_epochs):
    model.train()
    total = int(np.ceil(X_train.shape[0]/batch_size))
    with tqdm_notebook(total=total, leave=True) as prog_bar:
        
        for x, y in get_batches(X_train, Y_train, batch_size,
                                mode='train', use_gpu=use_gpu):
            output = model(x)
            loss = criterion(output, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            running_batch += 1
            prog_bar.set_description_str('Training loss after {} batches: {:.3f}'.format(
                        running_batch, running_loss/running_batch))
            prog_bar.update(1)
    valid_loss = get_eval_loss(model, criterion, X_valid,
                               Y_valid, batch_size, False)
    time.sleep(2)
    tqdm.write("Validation loss after epoch {}: {}".format(epoch, valid_loss))
    if valid_loss < min_loss:
        min_loss = valid_loss
        tqdm.write('Validation loss improved! Saving model.')
        with open(checkpoint, 'wb') as f:
            torch.save(model.state_dict(), f)
            running_patience = patience
    else:
        running_patience -= 1
    if running_patience == 0:
        tqdm.write('Ran out of patience, early stopping employed!')
        break


Validation loss after epoch 0: 0.12355968356132507
Validation loss improved! Saving model.



Validation loss after epoch 1: 0.07529342919588089
Validation loss improved! Saving model.



Validation loss after epoch 2: 0.047108471393585205
Validation loss improved! Saving model.



Validation loss after epoch 3: 0.026168327778577805
Validation loss improved! Saving model.



Validation loss after epoch 4: 0.009957388043403625
Validation loss improved! Saving model.



Validation loss after epoch 5: 0.003531928639858961
Validation loss improved! Saving model.


KeyboardInterrupt: 

In [11]:
bc

625

In [12]:
prog_bar.set_description?

In [9]:
model.load_state_dict(torch.load(checkpoint))

In [11]:
test_loss = get_eval_loss(model, criterion,
                          X_test, Y_test, batch_size, False)

In [12]:
test_loss

2.7139816284179688

In [13]:
test_preds, test_targets = get_eval_preds(model, X_test, Y_test, batch_size, False)

In [14]:
accuracy = accuracy_score(test_preds, test_targets)
accuracy

0.48

In [5]:
X_train.shape[0]

10000

In [9]:
int(np.ceil(10000/16))

625

In [15]:
prog_bar.clear?